In [1]:
import dfBasics
import common
import encoder
import pfAdapt
#import charts


In [2]:
import pandas as pd
from pyspark.sql import functions

# Lib functions

# Main

In [3]:
columns = ['CGLOBALMESSAGEID', 'CSTARTTIME', 'CENDTIME', 'CSTATUS', 'CSERVICE',\
       'CSLABILLINGMONTH', 'CSENDERPROTOCOL', 'CSENDERENDPOINTID',\
       'CINBOUNDSIZE', 'CRECEIVERPROTOCOL', 'CRECEIVERENDPOINTID', 'CSLATAT',\
       'CMESSAGETAT2', 'CSLADELIVERYTIME']
# withot 'CSLABILLINGMONTH'
def get_columns_2():
    columns = ['CGLOBALMESSAGEID', 'CSTARTTIME', 'CENDTIME', 'CSTATUS', 'CSERVICE',\
            'CSENDERPROTOCOL', 'CSENDERENDPOINTID',\
           'CINBOUNDSIZE', 'CRECEIVERPROTOCOL', 'CRECEIVERENDPOINTID', 'CSLATAT',\
           'CMESSAGETAT2', 'CSLADELIVERYTIME']
    return columns
#columns = get_columns_2()
#to count messages sent
columns = [ 'CSTARTTIME', 'CSENDERENDPOINTID']

In [4]:
sparkSession = dfBasics.getSparkSession()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/08/21 14:48:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/21 14:48:26 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [ ]:
df = sparkSession.read.parquet('hdfs://172.30.17.145:8020/sla_sql_data/*/*').select(columns).dropDuplicates() 

In [ ]:
df.write.parquet("/tmp/sla.parquet") 

In [ ]:
df = sparkSession.read.parquet("/tmp/sla.parquet")

In [ ]:
df.head()  

In [ ]:
# fix null endpoints
from pyspark.sql.functions import when
df2 = df.withColumn("CSENDERENDPOINTID", when(df.CSENDERENDPOINTID.isNull(), "None").otherwise(df.CSENDERENDPOINTID))

In [ ]:
df2.head()

In [ ]:
!rm -rf /tmp/sla.parquet
df2.write.parquet("/tmp/sla.parquet") 

In [ ]:
senders = df2.select([ 'CSENDERENDPOINTID']).dropDuplicates().toPandas() 

In [ ]:
senders.to_parquet('/tmp/senders' + '.parquet', engine='pyarrow', compression='GZIP')

In [ ]:
#senders[senders['CSENDERENDPOINTID'] == 'None']
!ls -l /tmp/senders.parquet

In [ ]:
import numpy as np
import encoder

def createEncoders(dataall,columns):
    for column in columns:
        le = encoder.TolerantLabelEncoder(ignore_unknown=True)
        #le.fit([1, 2, 2, 6])
        le.fit(dataall[column])
        encoder.LabelEncoder()
        print(le.classes_)
        np.save('/home/jovyan/work/cls/jupyter/npy/' + column + '.npy', le.classes_)
        
#!ls /home/jovyan/work/cls/jupyter/npy
createEncoders(senders,['CSENDERENDPOINTID'])

In [5]:
df2 = sparkSession.read.parquet("/tmp/sla.parquet")
#senders = pd.read_parquet('/tmp/senders' + '.parquet', engine='pyarrow')

In [6]:
import numpy as np
import encoder

np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

column = 'CSENDERENDPOINTID'
_encoder = encoder.TolerantLabelEncoder(ignore_unknown=True)
_encoder.classes_ = np.load('/home/jovyan/work/cls/jupyter/npy/' + column + '.npy')
#dataall[column] = _encoder.transform(dataall[column]) 

# restore np.load for future normal usage
#np.load = np_load_old

In [ ]:
_encoder.transform(['None'])
#!ls /home/jovyan/work/jupyter/npy/    
#!pwd

In [7]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def transform(value):
    #print(value)
    return str( _encoder.transform([value])[0])
    #return value
    #return '10' + value

udf_transform = udf(lambda z: transform(z), StringType())

#df2.withColumn("CSENDERENDPOINTID", str( _encoder.transform([df2.CSENDERENDPOINTID])[0])) 
df2 = df2.withColumn("CSENDERENDPOINTID", udf_transform(df2.CSENDERENDPOINTID)) 

In [ ]:
from pyspark.sql.types import IntegerType
df3 = df3.withColumn("CSENDERENDPOINTID", df3["CSENDERENDPOINTID"].cast(IntegerType()))

In [ ]:
#######
df3.head(11)


In [8]:
df2.write.parquet("/tmp/sla_enc.parquet")

KeyboardInterrupt: 

In [ ]:
pfall = df3.toPandas() 

In [ ]:
from datetime import time
import datetime as dt
import calendar
import pytz
de = pytz.timezone('Europe/Berlin')

# long timestamp
def date(x):
    return  dt.datetime.fromtimestamp(float(x) / 1e3, tz=de)


def adddatecolumns(data,pf,column) :
    data['year'] = pf[column].apply(lambda x: date(x).date().year)
    data['month'] = pf[column].apply(lambda x: date(x).date().month)
    data['day'] = pf[column].apply(lambda x: date(x).date().day)
    data['hour'] = pf[column].apply(lambda x: date(x).time().hour)
    data['minute'] = pf[column].apply(lambda x: date(x).time().minute)
    #data['second'] = pf[column].apply(lambda x: x.time().second)
    #data['microsecond'] = pf[column].apply(lambda x: x.time().microsecond)

def converttimestampcolumnn(pf,tsc) :
    pf[tsc] = pf[tsc].apply(lambda x: dt.datetime.fromtimestamp(float(x) / 1e3))

In [ ]:
def astype(pfall,selected,newtype):
    for each in selected:
        pfall[each] = pfall[each].astype(newtype)

In [ ]:
selected = [  'CSENDERENDPOINTID']

astype(pfall,selected,str) 
encoder.encode(pfall,selected)
#astype(pfall,['CSTARTTIME','CENDTIME','CSLATAT','CMESSAGETAT2','CSLADELIVERYTIME','CINBOUNDSIZE'] ,int) 
#del(pfall['CSLABILLINGMONTH'])
#pfall['CGLOBALMESSAGEID'] = pfall['CGLOBALMESSAGEID'].apply(hash)
pfall = pfall.drop_duplicates()

In [ ]:
# convert timestamp to datetime and add column date
#import calendar
##import pytz
#de = pytz.timezone('Europe/Berlin')
adddatecolumns(pfall,pfall,'CSTARTTIME')

In [ ]:
astype(pfall,['CSTARTTIME'] ,int) 

In [ ]:
import time; ts = str(time.time()) 
pfall.to_parquet('/tmp/msgsenders_' + ts + '.parquet', engine='fastparquet', compression='GZIP')

In [ ]:
#!mv /tmp/msgsenders_date.parquet /tmp/msgsenders_200716.parquet
import time; ts = time.time() 
print(ts)